### 0. Use Google Colab

In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
!pip install -r drive/MyDrive/CV_Transformer/requirements.txt
!pip install -U tensorflow-addons

### 1. Import libraries

In [29]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import matplotlib.pyplot as plt
from drive.MyDrive.CV_Transformer.src.ViT import ViT_classifier
from drive.MyDrive.CV_Transformer.src.BuildingBlocks import Patches

In [30]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

### 2. Data loading and preparation

In [31]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data(label_mode='fine')

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


In [32]:
(x_train, ys_train), (x_test, ys_test) = keras.datasets.cifar100.load_data(label_mode='coarse')

print(f"x_train shape: {x_train.shape} - y_train shape: {y_train.shape}")
print(f"x_test shape: {x_test.shape} - y_test shape: {y_test.shape}")

x_train shape: (50000, 32, 32, 3) - y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3) - y_test shape: (10000, 1)


### 3. Train ViT - classes

#### Hyperparameters

In [35]:
params = {
    "learning_rate" : 0.001,
    "weight_decay" : 0.0001,
    "MLP_dropout_rate" : 0.5,
    "TB_dropout_rate" : 0.6,
    "batch_size" : 256,
    "num_epochs" : 100,
    "input_shape" : (32, 32, 3),
    "num_classes" : 100,
    "image_size" : 72,  # We'll resize input images to this size
    "patch_size" : 6,  # Size of the patches to be extract from the input images
    "projection_dim" : 64,
    "num_heads" : 4,
    "transformer_units" : [128, 64], # Size of the transformer layers
    "transformer_layers" : 8,
    "mlp_head_units" : [2048, 1024]  # Size of the dense layers of the final classifier
    }

#### Training

In [36]:
ViT = ViT_classifier(x_train = x_train, params = params)
history = ViT.train(x_train=x_train, x_test=x_test, y_train=y_train, y_test=y_test)

Epoch 1/100
176/176 [==============================] - 90s 379ms/step - loss: 4.4470 - accuracy: 0.0495 - top-5-accuracy: 0.1694 - val_loss: 3.8262 - val_accuracy: 0.1164 - val_top-5-accuracy: 0.3194
Epoch 2/100
176/176 [==============================] - 65s 371ms/step - loss: 3.9097 - accuracy: 0.1016 - top-5-accuracy: 0.3019 - val_loss: 3.5314 - val_accuracy: 0.1646 - val_top-5-accuracy: 0.4238
Epoch 3/100
176/176 [==============================] - 65s 369ms/step - loss: 3.6598 - accuracy: 0.1354 - top-5-accuracy: 0.3765 - val_loss: 3.2881 - val_accuracy: 0.2102 - val_top-5-accuracy: 0.4922
Epoch 4/100
176/176 [==============================] - 65s 370ms/step - loss: 3.4913 - accuracy: 0.1638 - top-5-accuracy: 0.4272 - val_loss: 3.1532 - val_accuracy: 0.2414 - val_top-5-accuracy: 0.5266
Epoch 5/100
176/176 [==============================] - 65s 370ms/step - loss: 3.3440 - accuracy: 0.1915 - top-5-accuracy: 0.4690 - val_loss: 3.0599 - val_accuracy: 0.2560 - val_top-5-accuracy: 0.5456


#### Save model

In [37]:
# ViT.model.save("drive/MyDrive/CV_Transformer/ViT_model")

### 4. Train ViT - Superclasses

#### Hyperparameters

In [38]:
params_superclasses = params
params_superclasses["num_classes"] = 20

#### Training

In [39]:
ViT = ViT_classifier(x_train = x_train, params = params_superclasses)
history = ViT.train(x_train=x_train, x_test=x_test, y_train=ys_train, y_test=ys_test)

Epoch 1/100
176/176 [==============================] - 79s 385ms/step - loss: 2.8718 - accuracy: 0.1619 - top-5-accuracy: 0.4909 - val_loss: 2.4081 - val_accuracy: 0.2570 - val_top-5-accuracy: 0.6462
Epoch 2/100
176/176 [==============================] - 65s 371ms/step - loss: 2.4297 - accuracy: 0.2502 - top-5-accuracy: 0.6276 - val_loss: 2.1950 - val_accuracy: 0.3230 - val_top-5-accuracy: 0.7020
Epoch 3/100
176/176 [==============================] - 65s 371ms/step - loss: 2.2603 - accuracy: 0.3027 - top-5-accuracy: 0.6895 - val_loss: 2.0599 - val_accuracy: 0.3726 - val_top-5-accuracy: 0.7488
Epoch 4/100
176/176 [==============================] - 65s 372ms/step - loss: 2.1478 - accuracy: 0.3348 - top-5-accuracy: 0.7243 - val_loss: 1.9332 - val_accuracy: 0.4016 - val_top-5-accuracy: 0.7824
Epoch 5/100
176/176 [==============================] - 66s 373ms/step - loss: 2.0647 - accuracy: 0.3639 - top-5-accuracy: 0.7417 - val_loss: 1.8739 - val_accuracy: 0.4240 - val_top-5-accuracy: 0.7898


#### Save Model

In [ ]:
# ViT.model.save("drive/MyDrive/CV_Transformer/ViT_model_superclasses")

### 5. Examples

#### Show patches

In [ ]:
plt.figure(figsize=(4, 4))
example = np.random.choice(range(x_test.shape[0]))
image = x_test[example]
plt.imshow(image.astype("uint8"))
plt.axis("off")

print("Class:")
print(f"True: {fine_label_names[y_test[example,0]]}")
print(f"Predicted: {fine_label_names[np.argmax(model(image.reshape((1,32,32,3))))]}")
print("\n")
print("Superclass:")
print(f"True: {coarse_label_names[ys_test[example,0]]}")
print(f"Predicted: {coarse_label_names[np.argmax(model_superclasses(image.reshape((1,32,32,3))))]}")


#### Show results

In [ ]:
model = tf.keras.models.load_model("drive/MyDrive/CV_Transformer/ViT_model")
model_superclasses = tf.keras.models.load_model("drive/MyDrive/CV_Transformer/ViT_model_superclasses")

In [ ]:
file = "drive/MyDrive/CV_Transformer/cifar-100-python/meta"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

dict = unpickle(file)
fine_label_names = dict[b"fine_label_names"]
coarse_label_names = dict[b"coarse_label_names"]

In [ ]:
plt.figure(figsize=(4, 4))
example = np.random.choice(range(x_test.shape[0]))
image = x_test[example]
plt.imshow(image.astype("uint8"))
plt.axis("off")

print("Class:")
print(f"True: {fine_label_names[y_test[example,0]]}")
print(f"Predicted: {fine_label_names[np.argmax(model(image.reshape((1,32,32,3))))]}")
print("\n")
print("Superclass:")
print(f"True: {coarse_label_names[ys_test[example,0]]}")
print(f"Predicted: {coarse_label_names[np.argmax(model_superclasses(image.reshape((1,32,32,3))))]}")
